Cargamos las tablas en la base de datos de mysql con los achivos prosesados

In [3]:
import pandas as pd
import pymysql
from datetime import date, timedelta
import mysql.connector

Cargamos los datos de la tabla mysql  con los datos del archivo         

In [ ]:
# Conexión a la base de datos MySQL
conexion = pymysql.connect(
    host=", 
    user=", 
    password="", 
    database="Tecnologias_comunicaciones"
)
cursor = conexion.cursor()


# Crear tabla en MySQL para almacenar los datos del calendario
crear_tabla = """
CREATE TABLE IF NOT EXISTS Calendario (
    Fecha DATE PRIMARY KEY,
    Anio INT,
    Mes INT,
    Dia INT,
    Trimestre INT,
    Dia_Semana VARCHAR(20),
    Es_Fin_De_Semana BOOLEAN,
    Semana INT
);
"""
cursor.execute(crear_tabla)

# Función para generar fechas de inicio de cada trimestre
def generar_fechas_trimestrales(anio_inicio, anio_fin):
    fechas = []
    for anio in range(anio_inicio, anio_fin + 1):
        fechas.append(date(anio, 1, 1))  # 1 de enero
        fechas.append(date(anio, 4, 1))  # 1 de abril
        fechas.append(date(anio, 7, 1))  # 1 de julio
        fechas.append(date(anio, 10, 1)) # 1 de octubre
    return fechas

# Generamos fechas desde 2013 hasta 2030
fechas = generar_fechas_trimestrales(2013, 2030)

# Crear DataFrame de las fechas
df = pd.DataFrame({"Fecha": fechas})

# Convertir la columna Fecha a tipo datetime de pandas
df["Fecha"] = pd.to_datetime(df["Fecha"])

# Extraer las propiedades de las fechas
df["Anio"] = df["Fecha"].dt.year
df["Mes"] = df["Fecha"].dt.month
df["Dia"] = df["Fecha"].dt.day
df["Trimestre"] = df["Fecha"].dt.quarter
df["Dia_Semana"] = df["Fecha"].dt.day_name()
df["Es_Fin_De_Semana"] = df["Dia_Semana"].isin(["Saturday", "Sunday"])
df["Semana"] = df["Fecha"].dt.isocalendar().week  # Calcula el número de semana ISO

# Insertar datos en la tabla MySQL
insertar_calendario = """
INSERT INTO Calendario (Fecha, Anio, Mes, Dia, Trimestre, Dia_Semana, Es_Fin_De_Semana, Semana)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

for _, row in df.iterrows():
    cursor.execute(insertar_calendario, (
        row["Fecha"], row["Anio"], row["Mes"], row["Dia"], 
        row["Trimestre"], row["Dia_Semana"], row["Es_Fin_De_Semana"], row["Semana"]
    ))

# Confirmar los cambios en la base de datos
conexion.commit()
print("Datos insertados en la tabla Calendario.")

# Cerrar conexión
cursor.close()
conexion.close()

Datos insertados en la tabla Calendario.


Cargamos los datos de la tabla mysql Acceso sin Rango Internet con los datos del archivo  Acc_vel_loc_sinrangos.csv       

In [ ]:
# Configuración de conexión a MySQL
db_config = {
    "host": "",
    "user": ",
    "password": "
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Cargamos el archivo CSV
csv_file_path = r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Internet\Acc_vel_loc_sinrangos.csv'

# Establecemos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    # Cargamos el archivo CSV en un DataFrame
    df = pd.read_csv(csv_file_path)
    print("Columnas en el archivo CSV:", df.columns)

    # Renombramos la columna para que coincida con la tabla
    if 'Velocidad (Mbps)' in df.columns:
        df.rename(columns={'Velocidad (Mbps)': 'Velocidad_Mbps'}, inplace=True)

    # Insertamos los datos en la tabla
    insert_query = """
    INSERT INTO `Acceso sin Rango Internet` (
        `Id_Provincia`, `Provincia`, `Id_Localidad`, `Localidad`, 
        `Velocidad_Mbps`, `Accesos`
    ) VALUES (%s, %s, %s, %s, %s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertamos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Id_Provincia'],
            row['Provincia'],
            row['Id_Localidad'],
            row['Localidad'],
            row['Velocidad_Mbps'] if pd.notnull(row['Velocidad_Mbps']) else None,
            row['Accesos']
        ))

    # Guardamos los cambios
    connection.commit()

    # Verificamos los datos cargados
    cursor.execute("SELECT * FROM `Acceso sin Rango Internet` LIMIT 10;")
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos conexión
    cursor.close()
    connection.close()




Columnas en el archivo CSV: Index(['Id_Provincia', 'Provincia', 'Id_Localidad', 'Localidad',
       'Velocidad (Mbps)', 'Accesos'],
      dtype='object')
(1, 'BUENOS AIRES', 1, '25 de Mayo', Decimal('6854100.00'), 1)
(1, 'BUENOS AIRES', 1, '25 de Mayo', Decimal('6854100.00'), 2)
(1, 'BUENOS AIRES', 1, '25 de Mayo', Decimal('6854100.00'), 19)
(1, 'BUENOS AIRES', 1, '25 de Mayo', Decimal('6854100.00'), 85)
(1, 'BUENOS AIRES', 1, '25 de Mayo', Decimal('6854100.00'), 145)
(1, 'BUENOS AIRES', 1, '25 de Mayo', Decimal('6854100.00'), 76)
(1, 'BUENOS AIRES', 1, '25 de Mayo', Decimal('6854100.00'), 299)
(1, 'BUENOS AIRES', 1, '25 de Mayo', Decimal('6854100.00'), 288)
(1, 'BUENOS AIRES', 1, '25 de Mayo', Decimal('6854100.00'), 415)
(1, 'BUENOS AIRES', 1, '25 de Mayo', Decimal('6854100.00'), 1796)


Cargamos los datos de la tabla mysql Accesos_por_tecnologia_Internet  con los datos del archivo         

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Ruta del archivo CSV
csv_file_path = r'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/Internet/Accesos Por Tecnología.csv'

# Conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    # Leer el archivo CSV con pandas
    df = pd.read_csv(csv_file_path)
    
    # Reemplazar valores NaN por None explícitamente
    df = df.where(pd.notnull(df), None)
    
    # Verifica los datos del DataFrame
    
    
    # Crear consulta SQL para insertar datos
    insert_query = """
    INSERT INTO `Accesos_por_tecnologia_Internet` (
        `Id_Provincia`, `Provincia`, `Fecha`, 
        `ADSL`, `Cablemodem`, `Fibra_Optica`, `Wireless`, `Otros`, `Total`
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # Iterar sobre el DataFrame e insertar filas
    for _, row in df.iterrows():
        # Convertir valores individualmente a `None` si aún hay valores `NaN`
        data = [
            row['Id_Provincia'],
            row['Provincia'],
            row['Fecha'],
            row['ADSL'] if pd.notnull(row['ADSL']) else None,
            row['Cablemodem'] if pd.notnull(row['Cablemodem']) else None,
            row['Fibra óptica'] if pd.notnull(row['Fibra óptica']) else None,
            row['Wireless'] if pd.notnull(row['Wireless']) else None,
            row['Otros'] if pd.notnull(row['Otros']) else None,
            row['Total'] if pd.notnull(row['Total']) else None
        ]
        cursor.execute(insert_query, data)
    
    # Confirmar los cambios
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificar los datos cargados
    cursor.execute("SELECT * FROM `Accesos_por_tecnologia_Internet` LIMIT 10;")
    rows = cursor.fetchall()
   
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerrar conexión
    cursor.close()
    connection.close()



Datos cargados correctamente desde 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/Internet/Accesos Por Tecnología.csv'.
(1, 'Buenos Aires', datetime.date(2024, 6, 30), Decimal('214055.00'), Decimal('2722466.00'), Decimal('1849476.00'), Decimal('138638.00'), Decimal('64745.00'), Decimal('4989380.00'))
(2, 'Capital Federal', datetime.date(2024, 6, 30), Decimal('54102.00'), Decimal('1144781.00'), Decimal('230402.00'), Decimal('4493.00'), Decimal('29821.00'), Decimal('1463599.00'))
(3, 'Catamarca', datetime.date(2024, 6, 30), Decimal('4951.00'), Decimal('10303.00'), Decimal('58355.00'), Decimal('1384.00'), Decimal('81.00'), Decimal('75074.00'))
(4, 'Chaco', datetime.date(2024, 6, 30), Decimal('9448.00'), Decimal('57935.00'), Decimal('68944.00'), Decimal('8407.00'), Decimal('2358.00'), Decimal('147092.00'))
(5, 'Chubut', datetime.date(2024, 6, 30), Decimal('25955.00'), Decimal('80704.00'), Decimal('26516.00'), Decimal('31118.00'), Decimal('9930.00'), Decimal('174223.00'))
(6, 'Córdoba', dat

Cargamos los datos de la tabla mysql Accesos por rangos Internet con los datos del archivo  Accesos por rangos.csv'       

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Cargamos el archivo CSV
csv_file_path = r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Internet\Accesos por rangos.csv'

# Esatablesemos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    df = pd.read_csv(csv_file_path)
    
    # Rellenamos los datos  con valores nulos con 0
    df = df.where(pd.notnull(df), None)
    
    # Insertamos los datos en la tabla
    insert_query = """
    INSERT INTO `Accesos por rangos Internet` (
        `Id_Provincia`, `Provincia`, `Fecha`, 
        `HASTA_512_kbps`, `Mas_512Kbps_1Mbps`, `Mas_1Mbps_6Mbps`, 
        `Mas_6Mbps_10Mbps`, `Mas_10Mbps_20Mbps`, `Mas_20Mbps_30Mbps`, 
        `Mas_30Mbps`, `Otros`, `Total`
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertamos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Id_Provincia'],
            row['Provincia'],
            row['Fecha'],
            row['HASTA 512 kbps'] if pd.notnull(row['HASTA 512 kbps']) else None,
            row['+ 512 Kbps - 1 Mbps'] if pd.notnull(row['+ 512 Kbps - 1 Mbps']) else None,
            row['+ 1 Mbps - 6 Mbps'] if pd.notnull(row['+ 1 Mbps - 6 Mbps']) else None,
            row['+ 6 Mbps - 10 Mbps'] if pd.notnull(row['+ 6 Mbps - 10 Mbps']) else None,
            row['+ 10 Mbps - 20 Mbps'] if pd.notnull(row['+ 10 Mbps - 20 Mbps']) else None,
            row['+ 20 Mbps - 30 Mbps'] if pd.notnull(row['+ 20 Mbps - 30 Mbps']) else None,
            row['+ 30 Mbps'] if pd.notnull(row['+ 30 Mbps']) else None,
            row['OTROS'] if pd.notnull(row['OTROS']) else None,
            row['Total'] if pd.notnull(row['Total']) else None
        ))
    
    # Confirmamos los cambios
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificamos los datos cargados
    cursor.execute("SELECT * FROM `Accesos por rangos Internet` LIMIT 10;")
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos la conexión
    cursor.close()
    connection.close()


Datos cargados correctamente desde 'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Internet\Accesos por rangos.csv'.
Primeros 10 registros cargados:
(1, 'Buenos Aires', datetime.date(2024, 6, 30), Decimal('25287.00'), Decimal('23034.00'), Decimal('222627.00'), Decimal('221660.00'), Decimal('244528.00'), Decimal('106302.00'), Decimal('4020395.00'), Decimal('125547.00'), Decimal('4989380.00'))
(2, 'Capital Federal', datetime.date(2024, 6, 30), Decimal('518.00'), Decimal('4122.00'), Decimal('24539.00'), Decimal('40296.00'), Decimal('33158.00'), Decimal('5633.00'), Decimal('1355333.00'), Decimal('0.00'), Decimal('1463599.00'))
(3, 'Catamarca', datetime.date(2024, 6, 30), Decimal('72.00'), Decimal('164.00'), Decimal('2660.00'), Decimal('3093.00'), Decimal('3799.00'), Decimal('1279.00'), Decimal('63954.00'), Decimal('53.00'), Decimal('75074.00'))
(4, 'Chaco', datetime.date(2024, 6, 30), Decimal('236.00'), Decimal('349.00'), Decimal('14594.00'), Decimal('8336.00'), Decimal('6208.00'), Decimal(

Cargamos los datos de la tabla mysql Accesos_tecnologia_localidad_Internet con los datos del archivo  Accesos_tecnologia_localidad.csv       

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "",
    "user": ",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

#Cargamos el archivo CSV
csv_file_path = r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Internet\Accesos_tecnologia_localidad.csv'

# Establesemos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    df = pd.read_csv(csv_file_path)
    
    # Convertimos los valores no numéricos de 'Link Indec' a None
    df['Link Indec'] = pd.to_numeric(df['Link Indec'], errors='coerce')

    # # Rellenamos los datos  con valores nulos con 0
    df = df.where(pd.notnull(df), None)

    # Insertamos los datos en la tabla
    insert_query = """
    INSERT INTO `Accesos_tecnologia_localidad_Internet` (
        `Id_Provincia`, `Provincia`, `Id_Partido`, `Partido`, 
        `Id_Localidad`, `Localidad`, `Tecnologia`, `Link_Indec`, `Accesos`
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertamos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Id_Provincia'],
            row['Provincia'],
            row['Id_Partido'],
            row['Partido'],
            row['Id_Localidad'],
            row['Localidad'],
            row['Tecnologia'],
            row['Link Indec'] if pd.notnull(row['Link Indec']) else None,
            row['Accesos'] if pd.notnull(row['Accesos']) else None
        ))
    
    # Confirmamos los cambios
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificamos los datos cargados
    cursor.execute("SELECT * FROM `Accesos_tecnologia_localidad_Internet` LIMIT 10;")
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos la conexión
    cursor.close()
    connection.close()


Datos cargados correctamente desde 'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Internet\Accesos_tecnologia_localidad.csv'.
Primeros 10 registros cargados:
(1, 'BUENOS AIRES', 1, '25 de Mayo', 1, '25 de Mayo', 'ADSL', 6854100, Decimal('755.00'))
(1, 'BUENOS AIRES', 1, '25 de Mayo', 1, '25 de Mayo', 'CABLEMODEM', 6854100, Decimal('4600.00'))
(1, 'BUENOS AIRES', 1, '25 de Mayo', 1, '25 de Mayo', 'FIBRA OPTICA', 6854100, Decimal('2.00'))
(1, 'BUENOS AIRES', 1, '25 de Mayo', 1, '25 de Mayo', 'SATELITAL', 6854100, Decimal('742.00'))
(1, 'BUENOS AIRES', 1, '25 de Mayo', 1, '25 de Mayo', 'WIRELESS', 6854100, Decimal('727.00'))
(1, 'BUENOS AIRES', 1, '25 de Mayo', 2, 'Del Valle', 'ADSL', 6854020, Decimal('181.00'))
(1, 'BUENOS AIRES', 1, '25 de Mayo', 2, 'Del Valle', 'Otros', 6854020, Decimal('10.00'))
(1, 'BUENOS AIRES', 1, '25 de Mayo', 2, 'Del Valle', 'SATELITAL', 6854020, Decimal('1.00'))
(1, 'BUENOS AIRES', 1, '25 de Mayo', 3, 'Ernestina', 'ADSL', 6854030, Decimal('75.00'))
(1, 'BUENOS 

Cargamos los datos de la tabla mysql Dial_BAf_Internet con los datos del archivo  Dial-BAf.csv       

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Cargamos el archivo CSV
csv_file_path = r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Internet\Dial-BAf.csv'

# Establesemos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    df = pd.read_csv(csv_file_path)
    
    # Rellenamos los datos  con valores nulos con 0
    df = df.where(pd.notnull(df), None)
    
    # Query para insertar datos
    insert_query = """
    INSERT INTO Dial_BAf_Internet (
        Id_Provincia, Provincia, Fecha, 
        Banda_Ancha_Fija, Dial_Up, Total
    ) VALUES (%s, %s, %s, %s, %s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertamos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Id_Provincia'],
            row['Provincia'],
            row['Fecha'],
            row['Banda ancha fija'] if pd.notnull(row['Banda ancha fija']) else None,
            row['Dial up'] if pd.notnull(row['Dial up']) else None,
            row['Total'] if pd.notnull(row['Total']) else None
        ))
    
    # Confirmamos los cambios
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificamos los datos cargados
    cursor.execute("SELECT * FROM Dial_BAf_Internet LIMIT 10;")
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos la conexión
    cursor.close()
    connection.close()


Datos cargados correctamente desde 'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Internet\Dial-BAf.csv'.
Primeros 10 registros cargados:
(1, 'Buenos Aires', datetime.date(2024, 6, 30), 4983480, Decimal('5900.00'), 4989380)
(2, 'Capital Federal', datetime.date(2024, 6, 30), 1461549, Decimal('2050.00'), 1463599)
(3, 'Catamarca', datetime.date(2024, 6, 30), 75073, Decimal('1.00'), 75074)
(4, 'Chaco', datetime.date(2024, 6, 30), 147087, Decimal('5.00'), 147092)
(5, 'Chubut', datetime.date(2024, 6, 30), 173366, Decimal('857.00'), 174223)
(6, 'Córdoba', datetime.date(2024, 6, 30), 1085711, Decimal('63.00'), 1085774)
(7, 'Corrientes', datetime.date(2024, 6, 30), 156472, Decimal('3.00'), 156475)
(8, 'Entre Ríos', datetime.date(2024, 6, 30), 294291, Decimal('20.00'), 294311)
(9, 'Formosa', datetime.date(2024, 6, 30), 62950, Decimal('0.00'), 62950)
(10, 'Jujuy', datetime.date(2024, 6, 30), 119684, Decimal('0.00'), 119684)


Cargamos los datos de la tabla mysql Ingresos_Internet con los datos del archivo  Ingresos .csv       

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Cargamos el archivo CSV
csv_file_path = r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Internet\Ingresos .csv'

# Establesemos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    df = pd.read_csv(csv_file_path)
    
    # # Rellenamos los datos  con valores nulos con 0
    df = df.where(pd.notnull(df), None)
    
    # Query para insertar datos
    insert_query = """
    INSERT INTO Ingresos_Internet (Fecha, Ingresos_Miles_Pesos)
    VALUES (%s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertamos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Fecha'],
            row['Ingresos (miles de pesos)'] if pd.notnull(row['Ingresos (miles de pesos)']) else None
        ))
    
    # Confirmamos los cambios
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificamos los datos cargados
    cursor.execute("SELECT * FROM Ingresos_Internet LIMIT 10;")
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos la conexión
    cursor.close()
    connection.close()


Datos cargados correctamente desde 'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Internet\Ingresos .csv'.
Primeros 10 registros cargados:
(datetime.date(2024, 6, 30), Decimal('442032166.69'))
(datetime.date(2024, 3, 31), Decimal('346198986.13'))
(datetime.date(2023, 12, 31), Decimal('167376014.80'))
(datetime.date(2023, 9, 30), Decimal('133106593.41'))
(datetime.date(2023, 6, 30), Decimal('118060280.28'))
(datetime.date(2033, 3, 31), Decimal('104134992.81'))
(datetime.date(2022, 12, 31), Decimal('72443482.99'))
(datetime.date(2022, 9, 30), Decimal('64086784.33'))
(datetime.date(2022, 6, 30), Decimal('60419500.62'))
(datetime.date(2022, 3, 31), Decimal('55219313.01'))


Cargamos los datos de la tabla mysql Penetracion_Hogares_Internet con los datos del archivo  Penetracion-hogares.csv       

In [ ]:
# Configuramos de conexión a MySQL
db_config = {
    "host": "",
    "user": "r",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

#Cargamos el archivo CSV
csv_file_path = r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Internet\Penetracion-hogares.csv'

# Establesemos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    df = pd.read_csv(csv_file_path)
    
    # Rellenamos los datos  con valores nulos con 0
    df = df.where(pd.notnull(df), None)
    
    # Query para insertar datos
    insert_query = """
    INSERT INTO Penetracion_Hogares_Internet (Id_Provincia, Provincia, Fecha, Accesos_por_cada_100_hogares)
    VALUES (%s, %s, %s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertamos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Id_Provincia'],
            row['Provincia'],
            row['Fecha'],
            row['Accesos por cada 100 hogares'] if pd.notnull(row['Accesos por cada 100 hogares']) else None
        ))
    
    # Confirmamos los cambios
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificamos los datos cargados
    cursor.execute("SELECT * FROM Penetracion_Hogares_Internet LIMIT 10;")
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos conexión
    cursor.close()
    connection.close()


Datos cargados correctamente desde 'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Internet\Penetracion-hogares.csv'.
Primeros 10 registros cargados:
(1, 'Buenos Aires', datetime.date(2024, 6, 30), Decimal('79.84'))
(2, 'Capital Federal', datetime.date(2024, 6, 30), Decimal('116.37'))
(3, 'Catamarca', datetime.date(2024, 6, 30), Decimal('68.81'))
(4, 'Chaco', datetime.date(2024, 6, 30), Decimal('44.06'))
(5, 'Chubut', datetime.date(2024, 6, 30), Decimal('86.33'))
(6, 'Córdoba', datetime.date(2024, 6, 30), Decimal('90.79'))
(7, 'Corrientes', datetime.date(2024, 6, 30), Decimal('51.38'))
(8, 'Entre Ríos', datetime.date(2024, 6, 30), Decimal('68.64'))
(9, 'Formosa', datetime.date(2024, 6, 30), Decimal('39.69'))
(10, 'Jujuy', datetime.date(2024, 6, 30), Decimal('58.30'))


Cargamos los datos de la tabla mysql Penetración_poblacion_internet con los datos del archivo  Penetración-poblacion.csv       

In [ ]:
# Configuramos conexión a MySQL
db_config = {
    "host": "",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Cargamos el archivo CSV
csv_file_path = r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Internet\Penetración-poblacion.csv'

# Establesemos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    df = pd.read_csv(csv_file_path)
    
    # Rellenamos los datos  con valores nulos con 0
    df = df.where(pd.notnull(df), None)
    
    # Query para insertar datos
    insert_query = """
    INSERT INTO Penetración_poblacion_internet (Id_Provincia, Provincia, Fecha, Accesos_por_cada_100_hab)
    VALUES (%s, %s, %s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertamos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Id_Provincia'],
            row['Provincia'],
            row['Fecha'],
            row['Accesos por cada 100 hab'] if pd.notnull(row['Accesos por cada 100 hab']) else None
        ))
    
    # Confirmamos los cambios
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificamos los datos cargados
    cursor.execute("SELECT * FROM Penetración_poblacion_internet LIMIT 10;")
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos conexión
    cursor.close()
    connection.close()


Datos cargados correctamente desde 'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Internet\Penetración-poblacion.csv'.
Primeros 10 registros cargados:
(1, 'Buenos Aires', datetime.date(2024, 6, 30), Decimal('27.43'))
(2, 'Capital Federal', datetime.date(2024, 6, 30), Decimal('47.44'))
(3, 'Catamarca', datetime.date(2024, 6, 30), Decimal('17.50'))
(4, 'Chaco', datetime.date(2024, 6, 30), Decimal('11.78'))
(5, 'Chubut', datetime.date(2024, 6, 30), Decimal('26.46'))
(6, 'Córdoba', datetime.date(2024, 6, 30), Decimal('27.79'))
(7, 'Corrientes', datetime.date(2024, 6, 30), Decimal('13.53'))
(8, 'Entre Ríos', datetime.date(2024, 6, 30), Decimal('20.52'))
(9, 'Formosa', datetime.date(2024, 6, 30), Decimal('10.10'))
(10, 'Jujuy', datetime.date(2024, 6, 30), Decimal('14.91'))


Cargamos los datos de la tabla mysql Penetración_total_Intenet con los datos del archivo  Penetracion-totales.csv       

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Cargamos el archivo CSV
csv_file_path = r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Internet\Penetracion-totales.csv'

# Establesemos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    df = pd.read_csv(csv_file_path)
    
    # Rellenamos los datos  con valores nulos con 0
    df = df.where(pd.notnull(df), None)
    
    # Query para insertar datos
    insert_query = """
    INSERT INTO Penetración_total_Intenet (Fecha, Accesos_por_cada_100_hogares, Accesos_por_cada_100_hab)
    VALUES (%s, %s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertamos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Fecha'],
            row['Accesos por cada 100 hogares'] if pd.notnull(row['Accesos por cada 100 hogares']) else None,
            row['Accesos por cada 100 hab'] if pd.notnull(row['Accesos por cada 100 hab']) else None
        ))
    
    # Confirmamos los cambios
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificamos los datos cargados
    cursor.execute("SELECT * Penetración_total_Intenet LIMIT 10;")
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos conexión
    cursor.close()
    connection.close()


Datos cargados correctamente desde 'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Internet\Penetracion-totales.csv'.
Error: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Penetración_total_Intenet LIMIT 10' at line 1")


Cargamos los datos de la tabla mysql Total_Aceso_por_Rango_Internet con los datos del archivo   Totales Accesos por rango.csv      

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Cargamos la ruta del archivo CSV
csv_file_path = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\Internet\\Totales Accesos por rango.csv'

# Establesemos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    # Leer el archivo CSV con pandas
    df = pd.read_csv(csv_file_path)
    
    insert_query = """
    INSERT INTO Total_Aceso_por_Rango_Internet(
        Fecha, Hasta_512_kbps, Entre_512_Kbps_y_1_Mbps, Entre_1_Mbps_y_6_Mbps,
        Entre_6_Mbps_y_10_Mbps, Entre_10_Mbps_y_20_Mbps, Entre_20_Mbps_y_30_Mbps,
        Mas_de_30_Mbps, OTROS, Total
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertamos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Fecha'],
            row['Hasta 512 kbps'],
            row['Entre 512 Kbps y 1 Mbps'],
            row['Entre 1 Mbps y 6 Mbps'],
            row['Entre 6 Mbps y 10 Mbps'],
            row['Entre 10 Mbps y 20 Mbps'],
            row['Entre 20 Mbps y 30 Mbps'],
            row['Más de 30 Mbps'],
            row['OTROS'],
            row['Total']
        ))
    
    # Confirmamos los cambios
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificalos los datos cargados
    cursor.execute("SELECT * FROM Total_Aceso_por_Rango_Internet LIMIT 10;")
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos la  conexión
    cursor.close()
    connection.close()


Datos cargados correctamente desde 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\Internet\\Totales Accesos por rango.csv'.
Primeros 10 registros cargados:
(datetime.date(2024, 6, 30), 28151, 67024, 840200, 911374, 662649, 348253, 8357088, 341368, 11556107)
(datetime.date(2024, 3, 31), 28801, 69355, 866152, 950930, 672155, 353896, 8363694, 328173, 11633156)
(datetime.date(2023, 12, 31), 29708, 71742, 900253, 978108, 697232, 350290, 8224736, 295592, 11547661)
(datetime.date(2023, 9, 30), 30274, 80609, 945608, 998725, 720084, 350918, 8003269, 277421, 11406908)
(datetime.date(2023, 6, 30), 30827, 79405, 963510, 1028250, 734125, 364730, 7775095, 275955, 11251897)
(datetime.date(2023, 3, 31), 31437, 82048, 1001553, 1060321, 729194, 357769, 7564819, 414754, 11241895)
(datetime.date(2022, 12, 31), 31981, 85668, 1075272, 1109421, 765015, 335573, 7410902, 394282, 11208114)
(datetime.date(2022, 9, 30), 33013, 96727, 1161370, 1151906, 823505, 479822, 6995750, 349035, 11091128)
(datetime.date

Cargamos los datos de la tabla mysql  INSERT INTO Total_Accesos_Por_Tecnologia_Internet con los datos del archivo Totales Accesos Por Tecnología.csv 

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Cargamos el archivo CSV
csv_file_path = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\Internet\\Totales Accesos Por Tecnología.csv'

# Establesemos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    df = pd.read_csv(csv_file_path)
    
    # Insertamos los datos en la tabla
    insert_query = """
    INSERT INTO Total_Accesos_Por_Tecnologia_Internet (
        Fecha, ADSL, Cablemodem, Fibra_Optica, Wireless, Otros, Total
    ) VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertamos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Fecha'],
            row['ADSL'],
            row['Cablemodem'],
            row['Fibra óptica'],
            row['Wireless'],
            row['Otros'],
            row['Total']
        ))
    
    # Confirmamos los cambios
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificamos los datos cargados
    cursor.execute("SELECT * FROM Total_Accesos_Por_Tecnologia_Internet LIMIT 10;")
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos la conexión
    cursor.close()
    connection.close()


Datos cargados correctamente desde 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\Internet\\Totales Accesos Por Tecnología.csv'.
Primeros 10 registros cargados:
(datetime.date(2024, 6, 30), 733491, 5867504, 4169958, 593197, 191957, 11556107)
(datetime.date(2024, 3, 31), 774475, 5986957, 4015101, 598682, 257941, 11633156)
(datetime.date(2023, 12, 31), 836390, 6022532, 3908183, 585760, 194796, 11547661)
(datetime.date(2023, 9, 30), 897895, 6018832, 3708718, 581436, 200027, 11406908)
(datetime.date(2023, 6, 30), 1006509, 5997149, 3463988, 581823, 202428, 11251897)
(datetime.date(2023, 3, 31), 1204417, 6002582, 3243411, 576647, 214838, 11241895)
(datetime.date(2022, 12, 31), 1315373, 6037457, 3060575, 572969, 221740, 11208114)
(datetime.date(2022, 9, 30), 1395277, 6031970, 2871541, 557110, 235230, 11091128)
(datetime.date(2022, 6, 30), 1468333, 5979214, 2723285, 556243, 231609, 10958684)
(datetime.date(2022, 3, 31), 1533240, 6073426, 2219533, 545814, 251996, 10624009)


Cargamos los datos de la tabla mysql Totales_VMD_internet con los datos del archivo         

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Cargamos el archivo CSV
csv_file_path = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\Internet\\Totales VMD.csv'

# Establecemos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    # Cargamos el archivo CSV en un DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Renombramos las columnas si es necesario para que coincidan con la tabla SQL
    df.rename(columns={
        'Año': 'Año',
        'Trimestre': 'Trimestre',
        'Mbps (Media de bajada)': 'Mbps_Media_Bajada',
        'Trimestre.1': 'Periodo'
    }, inplace=True)
    
    # Insertamos los datos en la tabla
    insert_query = """
    INSERT INTO Totales_VMD_internet(
        Año, Trimestre, Mbps_Media_Bajada, Periodo
    ) VALUES (%s, %s, %s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertamos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Año'],
            row['Trimestre'],
            round(row['Mbps_Media_Bajada'], 2),  
            row['Periodo']
        ))
    
    # Confirmamos los cambios realizados
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificamos los datos cargados
    cursor.execute("SELECT * FROM Totales_VMD_internet LIMIT 10;")  
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos conexión
    cursor.close()
    connection.close()



Datos cargados correctamente desde 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\Internet\\Totales VMD.csv'.
Primeros 10 registros cargados:
(2024, 2, Decimal('139.25'), 'Abr-Jun 2024')
(2024, 1, Decimal('139.15'), 'Ene-Mar 2024')
(2023, 4, Decimal('139.04'), 'Oct-Dic 2023')
(2023, 3, Decimal('129.67'), 'Jul-Sept 2023')
(2023, 2, Decimal('123.95'), 'Abr-Jun 2023')
(2023, 1, Decimal('110.50'), 'Ene-Mar 2023')
(2022, 4, Decimal('106.56'), 'Oct-Dic 2022')
(2022, 3, Decimal('62.46'), 'Jul-Sept 2022')
(2022, 2, Decimal('58.44'), 'Abr-Jun 2022')
(2022, 1, Decimal('55.11'), 'Ene-Mar 2022')


Cargamos los datos de la tabla mysql Velocidad_por_Provincia_internet con los datos del archivo  Velocidad % por prov.csv'       

In [ ]:
#Configuramos la conexión a MySQL
db_config = {
    "host": "t",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Cargamos el archivo CSV
csv_file_path = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\Internet\\Velocidad % por prov.csv'

# Establesemos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    df = pd.read_csv(csv_file_path)
    
    # Renombramos las columnas si es necesario para que coincidan con la tabla SQL
    df.rename(columns={
        'Año': 'Año',
        'Trimestre': 'Trimestre',
        'Provincia': 'Provincia',
        'Mbps (Media de bajada)': 'Mbps_Media_Bajada'
    }, inplace=True)
    
    # Insertamos  los datos en la tabla
    insert_query = """
    INSERT INTO Velocidad_por_Provincia_internet (
        Año, Trimestre, Provincia, Mbps_Media_Bajada
    ) VALUES (%s, %s, %s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertamos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Año'],
            row['Trimestre'],
            row['Provincia'],
            round(row['Mbps_Media_Bajada'], 2)  
        ))
    
    # Confirmamos los cambios
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificamos si los datos se cargados bien
    cursor.execute("SELECT * FROM Velocidad_por_Provincia_internet LIMIT 10;")
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos conexión
    cursor.close()
    connection.close()


Datos cargados correctamente desde 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\Internet\\Velocidad % por prov.csv'.
Primeros 10 registros cargados:
(2024, 2, 'Buenos Aires', Decimal('157.41'))
(2024, 2, 'Capital Federal', Decimal('233.01'))
(2024, 2, 'Catamarca', Decimal('97.38'))
(2024, 2, 'Chaco', Decimal('107.76'))
(2024, 2, 'Chubut', Decimal('21.67'))
(2024, 2, 'Córdoba', Decimal('117.39'))
(2024, 2, 'Corrientes', Decimal('104.26'))
(2024, 2, 'Entre Ríos', Decimal('100.98'))
(2024, 2, 'Formosa', Decimal('82.66'))
(2024, 2, 'Jujuy', Decimal('49.87'))


Cargamos los datos de la tabla Velosidad_sin_Rango_internet  con los datos del archivo  Velocidad_sin_Rangos.csv       

In [ ]:
# Configuramos de conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Cargamos la ruta del archivo CSV
csv_file_path = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\Internet\\Velocidad_sin_Rangos.csv'

# Establesemos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    df = pd.read_csv(csv_file_path)
    
    ## Rellenamos los datos  con valores nulos con 0
    df.fillna({'Velocidad': 0, 'Accesos': 0}, inplace=True)
    
    # Insertamos los datos en la tabla
    insert_query = """
    INSERT INTO Velosidad_sin_Rango_internet (
        Fecha, Provincia, Velocidad, Accesos, Id_Provincia
    ) VALUES (%s, %s, %s, %s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertamos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Fecha'],
            row['Provincia'],
            row['Velocidad'],
            row['Accesos'],
            row['Id_Provincia']
        ))
    
    # Confirmamos los cambios
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificamos si los  datos  se cargaron
    cursor.execute("SELECT * FROM Velosidad_sin_Rango_internet LIMIT 10;")
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos la conexión
    cursor.close()
    connection.close()




Datos cargados correctamente desde 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\Internet\\Velocidad_sin_Rangos.csv'.
Primeros 10 registros cargados:
(datetime.date(2024, 6, 30), 'BUENOS AIRES', Decimal('75.00'), 1062, 1)
(datetime.date(2024, 6, 30), 'BUENOS AIRES', Decimal('59.00'), 59, 1)
(datetime.date(2024, 6, 30), 'BUENOS AIRES', Decimal('480.00'), 5, 1)
(datetime.date(2024, 6, 30), 'BUENOS AIRES', Decimal('3.50'), 41735, 1)
(datetime.date(2024, 6, 30), 'BUENOS AIRES', Decimal('18.00'), 1042, 1)
(datetime.date(2024, 6, 30), 'BUENOS AIRES', Decimal('9.00'), 750, 1)
(datetime.date(2024, 6, 30), 'BUENOS AIRES', Decimal('17.00'), 2, 1)
(datetime.date(2024, 6, 30), 'BUENOS AIRES', Decimal('8.00'), 63727, 1)
(datetime.date(2024, 6, 30), 'BUENOS AIRES', Decimal('105.00'), 14, 1)
(datetime.date(2024, 6, 30), 'BUENOS AIRES', Decimal('500.00'), 137199, 1)


Cargamos los datos de la tabla mysql mapa_de_conectividad_internet  con los datos del archivo  mapa_conectividadcsv       

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Establesemos la ruta  del archivo CSV
csv_file_path = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\mapa_conectividad\\mapa_conectividad.csv'

# Realisamos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    df = pd.read_csv(csv_file_path)
    
    # Rellenamos los datos  con valores nulos con 0
    df.fillna('No', inplace=True)  # 'No' para valores de conectividad ausentes
    df['Link'] = pd.to_numeric(df['Link'], errors='coerce').fillna(0)  # Reemplazar valores nulos en 'Link' con 0
    df['Latitud'] = pd.to_numeric(df['Latitud'], errors='coerce').fillna(0)
    df['Longitud'] = pd.to_numeric(df['Longitud'], errors='coerce').fillna(0)

    # Insertamos los datos en la tabla
    insert_query = """
    INSERT INTO mapa_de_conectividad_internet (
        Id_Provincia, Provincia, Id_Partido, Partido, Id_Localidad, Localidad,
        ADSL, Cablemodem, Dial_Up, Fibra_Optica, Satelital, Wireless,
        Telefonia_Fija, TresG, CuatroG, Link, Latitud, Longitud
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertamos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Id_Provincia'], row['Provincia'], row['Id_Partido'], row['Partido'], row['Id_Localidad'], row['Localidad'],
            row['ADSL'], row['Cablemódem'], row['Dial Up'], row['Fibra óptica'], row['Satelital'], row['Wireless'],
            row['Telefonía Fija'], row['3G'], row['4G'], row['Link'], row['Latitud'], row['Longitud']
        ))
    
    # Confirmamos si se realizaron  los cambios
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificamos los datos cargados
    cursor.execute("SELECT * FROM mapa_de_conectividad_internet LIMIT 10;")
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos la  conexión
    cursor.close()
    connection.close()


C:\Users\jrgsi\AppData\Local\Temp\ipykernel_20684\3104521016.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'No' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('No', inplace=True)  # 'No' para valores de conectividad ausentes


Datos cargados correctamente desde 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\mapa_conectividad\\mapa_conectividad.csv'.
Primeros 10 registros cargados:
(1, 'BUENOS AIRES', 1, '25 de Mayo', 1, '25 de Mayo', 'SI', 'SI', 'No', 'No', 'SI', 'SI', 'SI', 'SI', 'SI', Decimal('6854100.00'), Decimal('-35.4339385893'), Decimal('-60.1731209454'))
(1, 'BUENOS AIRES', 1, '25 de Mayo', 2, 'Agustín Mosconi', 'No', 'No', 'No', 'No', 'No', 'No', 'SI', 'No', 'No', Decimal('6854010.00'), Decimal('-35.7392169542'), Decimal('-60.5590634622'))
(1, 'BUENOS AIRES', 1, '25 de Mayo', 3, 'Del Valle', 'SI', 'No', 'No', 'No', 'SI', 'No', 'No', 'SI', 'No', Decimal('6854020.00'), Decimal('-35.8973453888'), Decimal('-60.7316089513'))
(1, 'BUENOS AIRES', 1, '25 de Mayo', 4, 'Ernestina', 'SI', 'No', 'No', 'No', 'No', 'No', 'SI', 'No', 'No', Decimal('6854030.00'), Decimal('-35.2702340520'), Decimal('-59.5575034079'))
(1, 'BUENOS AIRES', 1, '25 de Mayo', 5, 'Gobernador Ugarte', 'No', 'No', 'No', 'No', 'No', 'SI'

Cargamos los datos de la tabla Telefonia_Fija_Acceso_total  con los datos del archivo   Fija_accesos_tot.csv     

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Cargamos el archivo CSV
csv_file_path = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\telefonia_fija\\Fija_accesos_tot.csv'

# Establesemos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
 
    df = pd.read_csv(csv_file_path)
    
    # # Rellenamos los datos  con valores nulos con 0
    df.fillna(0, inplace=True)
    
    # Insertamos los datos en la tabla
    insert_query = """
    INSERT INTO Telefonia_Fija_Acceso_total (
        Fecha, Accesos_Telefonia_Fija_Total, Accesos_Telefonia_Fija_Hogares,
        Accesos_Telefonia_Fija_Comercial, Accesos_Telefonia_Fija_Gobierno,
        Accesos_Telefonia_Fija_Otros
    ) VALUES (%s, %s, %s, %s, %s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertamos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Fecha'],
            row['Accesos telefonía fija (total)'],
            row['Accesos telefonía fija (hogares)'],
            row['Accesos telefonía fija (comercial)'],
            row['Accesos telefonía fija (gobierno)'],
            row['Accesos telefonía fija (otros)']
        ))
    
    # Confirmamos los cambios realisados
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificamos los datos cargados
    cursor.execute("SELECT * FROM Telefonia_Fija_Acceso_total LIMIT 10;")
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos la  conexión
    cursor.close()
    connection.close()


Datos cargados correctamente desde 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\telefonia_fija\\Fija_accesos_tot.csv'.
Primeros 10 registros cargados:
(datetime.date(2014, 3, 31), 9562231, 7897980, 1065834, 30917, 567500)
(datetime.date(2014, 6, 30), 9548994, 7872511, 1073038, 31339, 572106)
(datetime.date(2014, 9, 30), 9613361, 7924613, 1091747, 32516, 564485)
(datetime.date(2014, 12, 31), 9689752, 7989623, 1106502, 33139, 560488)
(datetime.date(2015, 3, 31), 10086221, 8358053, 1134427, 36888, 556853)
(datetime.date(2015, 6, 30), 9991190, 8291156, 1103025, 34419, 562590)
(datetime.date(2015, 9, 30), 10050617, 8364541, 1095767, 35230, 555079)
(datetime.date(2015, 12, 31), 9975798, 8380800, 1003363, 35161, 556474)
(datetime.date(2016, 3, 31), 10239381, 8950226, 721226, 37841, 530088)
(datetime.date(2016, 6, 30), 10379749, 9037000, 759247, 38093, 545409)


Cargamos los datos de la tabla Telefonia_Fija_Ingresos con los datos del archivo  Fija_ingresos.csv     

In [ ]:
# Configuración de conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Ruta del archivo CSV
csv_file_path = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\telefonia_fija\\Fija_ingresos.csv'

# Conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    # Leer el archivo CSV con pandas
    df = pd.read_csv(csv_file_path)
    
    # Limpiar datos: asegurar que no haya valores nulos
    df.fillna(0, inplace=True)
    
    # Insertar los datos en la tabla
    insert_query = """
    INSERT INTO Telefonia_Fija_Ingresos (
        Fecha, Ingresos_Miles_Pesos
    ) VALUES (%s, %s)
    """
    
    # Iterar sobre el DataFrame e insertar filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Fecha'],
            row['Ingresos (miles de $)']
        ))
    
    # Confirmar los cambios
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificar datos cargados
    cursor.execute("SELECT * FROM Telefonia_Fija_Ingresos LIMIT 10;")
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerrar conexión
    cursor.close()
    connection.close()


Datos cargados correctamente desde 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\telefonia_fija\\Fija_ingresos.csv'.
Primeros 10 registros cargados:
(datetime.date(2014, 3, 31), Decimal('2916071.754'))
(datetime.date(2014, 6, 30), Decimal('2974865.204'))
(datetime.date(2014, 9, 30), Decimal('3105462.022'))
(datetime.date(2014, 12, 31), Decimal('3339690.895'))
(datetime.date(2015, 3, 31), Decimal('3260124.752'))
(datetime.date(2015, 6, 30), Decimal('3425518.480'))
(datetime.date(2015, 9, 30), Decimal('3590883.181'))
(datetime.date(2015, 12, 31), Decimal('3815364.119'))
(datetime.date(2016, 3, 31), Decimal('4198260.392'))
(datetime.date(2016, 6, 30), Decimal('4718769.816'))


Cargamos los datos de la tabla Telefonia_Fija_Penetracion_Provincias  con los datos del archivo    Fija_penetracion_prov

In [ ]:
# Configuración de conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Cargamos archivo CSV
csv_file_path = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\telefonia_fija\\Fija_penetracion_prov.csv'

# Establesemos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    
    df = pd.read_csv(csv_file_path)
    
    # Rellenamos los datos  con valores nulos con 0s
    df.fillna(0, inplace=True)
    
    #Insertamos los datos fila por fila
    insert_query = """
    INSERT INTO Telefonia_Fija_Penetracion_Provincias (
        Id_Provincia, Provincia, Fecha,
        Accesos_Telefonia_Fija_por_100_Hab, Accesos_Telefonia_Fija_por_100_Hogares
    ) VALUES (%s, %s, %s, %s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertarmos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Id_Provincia'],
            row['Provincia'],
            row['Fecha'],
            row['Accesos telefonía fija por cada 100 hab'],
            row['Accesos telefonía fija por cada 100 hogares']
        ))
    
    # Confirmamos los cambios
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificamos los datos cargados
    cursor.execute("SELECT * FROM Telefonia_Fija_Penetracion_Provincias LIMIT 10;")
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos la conexión
    cursor.close()
    connection.close()


Datos cargados correctamente desde 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\telefonia_fija\\Fija_penetracion_prov.csv'.
Primeros 10 registros cargados:
(1, 'Buenos Aires', datetime.date(2014, 3, 31), Decimal('26.717541'), Decimal('85.264880'))
(2, 'Capital Federal', datetime.date(2014, 3, 31), Decimal('52.663108'), Decimal('128.570341'))
(3, 'Catamarca', datetime.date(2014, 3, 31), Decimal('10.034940'), Decimal('39.446596'))
(4, 'Chaco', datetime.date(2014, 3, 31), Decimal('9.094093'), Decimal('34.022053'))
(5, 'Chubut', datetime.date(2014, 3, 31), Decimal('17.657914'), Decimal('57.594630'))
(6, 'Córdoba', datetime.date(2014, 3, 31), Decimal('21.091643'), Decimal('68.884254'))
(7, 'Corrientes', datetime.date(2014, 3, 31), Decimal('9.713546'), Decimal('36.884951'))
(8, 'Entre Ríos', datetime.date(2014, 3, 31), Decimal('17.141516'), Decimal('57.327000'))
(9, 'Formosa', datetime.date(2014, 3, 31), Decimal('7.377900'), Decimal('28.984601'))
(10, 'Jujuy', datetime.date(2014, 3, 3

Cargamos los datos en la tabla Telefonia_Fija_Penetracion_Total con los datos del archivo Fija_penetracion_tot.csv

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones",
    "charset": "utf8mb4"
}

# Cargamos el archivo CSV
csv_file_path = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\telefonia_fija\\Fija_penetracion_tot.csv'

# Establecemos la conexión a la base de datos
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

try:
    # Cargamos el archivo CSV en un DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Rellenamos valores nulos con 0
    df.fillna(0, inplace=True)
    
    # Insertamos los datos en la tabla
    insert_query = """
    INSERT INTO Telefonia_Fija_Penetracion_Total (
        Fecha,
        Accesos_Telefonia_Fija_por_100_Hab,
        Accesos_Telefonia_Fija_por_100_Hogares
    ) VALUES (%s, %s, %s)
    """
    
    # Iteramos sobre el DataFrame e insertamos las filas
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Fecha'],
            row['Accesos telefonía fija por cada 100 hab'],
            row['Accesos telefonía fija por cada 100 hogares']
        ))
    
    # Confirmamos los cambios
    connection.commit()
    print(f"Datos cargados correctamente desde '{csv_file_path}'.")
    
    # Verificamos los datos cargados
    cursor.execute("SELECT * FROM Telefonia_Fija_Penetracion_Total LIMIT 10;")
    rows = cursor.fetchall()
    print("Primeros 10 registros cargados:")
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")
finally:
    # Cerramos la conexión
    cursor.close()
    connection.close()



Datos cargados correctamente desde 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\telefonia_fija\\Fija_penetracion_tot.csv'.
Primeros 10 registros cargados:
(datetime.date(2024, 6, 30), Decimal('14.291425'), Decimal('45.442151'))
(datetime.date(2024, 3, 31), Decimal('14.719928'), Decimal('46.848116'))
(datetime.date(2023, 12, 31), Decimal('15.440544'), Decimal('49.696153'))
(datetime.date(2023, 9, 30), Decimal('15.694047'), Decimal('50.552590'))
(datetime.date(2023, 6, 30), Decimal('15.809957'), Decimal('50.965804'))
(datetime.date(2023, 3, 31), Decimal('15.504410'), Decimal('50.019890'))
(datetime.date(2022, 12, 31), Decimal('15.455165'), Decimal('49.743212'))
(datetime.date(2022, 9, 30), Decimal('15.456187'), Decimal('49.786414'))
(datetime.date(2022, 6, 30), Decimal('15.566152'), Decimal('50.179860'))
(datetime.date(2022, 3, 31), Decimal('15.969679'), Decimal('51.520930'))


Cargamos los datos en la tabla Telefonia_Fija_Provincia_Tipo  con los datos del archivo Fija_prov_tipo.csv


In [ ]:
# Cargamos el archivo CSV
file_path = r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\telefonia_fija\Fija_prov_tipo.csv'
data = pd.read_csv(file_path)

# cambiamos los valores nulos por 0
data.fillna(0, inplace=True)

# Nos conectar a la base de datos MySQL
connection = mysql.connector.connect(
    host="",
    user="",
    password="",
    database="Tecnologias_comunicaciones"
)

cursor = connection.cursor()

# Query para insertar datos
insert_query = """
INSERT INTO Telefonia_Fija_Provincia_Tipo (
    Fecha, Provincia, Accesos_telefonia_fija_total,
    Accesos_telefonia_fija_hogares, Accesos_telefonia_fija_comercial,
    Accesos_telefonia_fija_gobierno, Accesos_telefonia_fija_otros, Id_Provincia
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insertamos datos fila por fila
for _, row in data.iterrows():
    cursor.execute(insert_query, (
        row['Fecha'],
        row['Provincia'],
        int(row['Accesos telefonía fija (total)']),
        int(row['Accesos telefonía fija (hogares)']),
        int(row['Accesos telefonía fija (comercial)']),
        int(row['Accesos telefonía fija (gobierno)']),
        int(row['Accesos telefonía fija (otros)']),
        int(row['Id_Provincia'])
    ))

# Confirmamos  los cambios y cerramos la  conexión
connection.commit()
cursor.close()
connection.close()


Cargamos los datos en la tabla Telefonia_Movil_Accesos  con los datos del archivo Accesos.csv

In [ ]:
# Cargamos el archivo CSV
file_path = r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Telefonia_movil\Accesos.csv'
data = pd.read_csv(file_path)

# Rellenamos los datos con valores nulos con 0
data.fillna(0, inplace=True)

# Nos conectamos con la base de datos MySQL
connection = mysql.connector.connect(
    host="",
    user="t",
    password="",
    database="Tecnologias_comunicaciones"
)

cursor = connection.cursor()

# Query para insertar datos
insert_query = """
INSERT INTO Telefonia_Movil_Accesos  (
    Fecha, Total_accesos_pospago, Total_accesos_prepago,
    Total_accesos_operativos, Periodo
) VALUES (%s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    Total_accesos_pospago = VALUES(Total_accesos_pospago),
    Total_accesos_prepago = VALUES(Total_accesos_prepago),
    Total_accesos_operativos = VALUES(Total_accesos_operativos),
    Periodo = VALUES(Periodo);
"""

# Insertamos los datos fila por fila
for _, row in data.iterrows():
    cursor.execute(insert_query, (
        row['Fecha'],
        float(row['Total de accesos pospago']),
        float(row['Total de accesos prepago']),
        int(row['Total de accesos operativos']),
        row['Periodo']
    ))

# Confirmamos los cambios y cerramos la conexión
connection.commit()
cursor.close()
connection.close()

print("Datos insertados correctamente en la tabla 'Telefonia_Movil_Accesos '.")




Datos insertados correctamente en la tabla 'Telefonia_Movil_Accesos '.


Cargamos los datos de la tabla mysql Telefonia_movil_Ingresos con los datos del archivo         

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones"
}

# Cargamos la ruta del archivo CSV
csv_file_path = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\Telefonia_movil\\Ingresos.csv'

# Cargamos el archivo CSV
data = pd.read_csv(csv_file_path)

# Establesemos la conexión a la base de datos
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

# Query para insertar datos
insert_query = """
INSERT INTO Telefonia_movil_Ingresos (Fecha, Ingresos_Miles_Dol)
VALUES (%s, %s)
"""

# Iteramos sobre el DataFrame e insertamos los datos y  Convertimos los ingresos a entero
for _, row in data.iterrows():
    cursor.execute(insert_query, (
        row['Fecha'],
        int(row['Ingresos (miles de $)'])  
    ))

# Confirmamos los cambios y cerramos la conexión
connection.commit()
cursor.close()
connection.close()

print("Datos insertados correctamente en la tabla 'Telefonia_movil_Ingresos'.")


Datos insertados correctamente en la tabla 'Telefonia_movil_Ingresos'.


Cargamos los datos de la tabla mysql Telefonia_Movil_Llamadas_salientes con los datos del archivo  Llamadas salientes.csv      

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones"
}

# Cargamos la ruta del archivo CSV
csv_file_path = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\Telefonia_movil\\Llamadas salientes.csv'

# Cargamos el archivo CSV
data = pd.read_csv(csv_file_path)

# Conexión a la base de datos
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

# Query para insertar datos
insert_query = """
INSERT INTO Telefonia_Movil_Llamadas_salientes (
    Fecha,
    Llamadas_pospago_salientes_miles,
    Llamadas_prepago_salientes_miles,
    Total_llamadas_salientes_miles
) VALUES (%s, %s, %s, %s)
"""

# Iteramos sobre el DataFrame e insertamos los datos
for _, row in data.iterrows():
    cursor.execute(insert_query, (
        row['Fecha'],
        float(row['Llamadas pospago salientes (miles)']),
        float(row['Llamadas prepago salientes (miles)']),
        float(row['Total de llamadas salientes (miles)'])
    ))

# Confirmamos los cambios y cerramos la conexión
connection.commit()
cursor.close()
connection.close()

print("Datos insertados correctamente en la tabla 'Telefonia_Movil_Llamadas_salientes'.")


Datos insertados correctamente en la tabla 'Telefonia_Movil_Llamadas_salientes'.


Cargamos los datos de la tabla mysql Minutos_salientes_Tel_Movil con los datos del archivo  Minutos salientes.csv       

In [ ]:
# Configuramos la  conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones"
}

#Cargamos la ruta del archivo CSV
csv_file_path = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\Telefonia_movil\\Minutos salientes.csv'

# Cargamos el archivo CSV
data = pd.read_csv(csv_file_path)

# Establesemos la conexión a la base de datos
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

# Query para insertar datos
insert_query = """
INSERT INTO Minutos_salientes_Tel_Movil (
    Fecha,
    Minutos_pospago_salientes_miles,
    Minutos_prepago_salientes_miles,
    Total_minutos_salientes_miles
) VALUES (%s, %s, %s, %s)
"""

# Iteramos sobre el DataFrame e insertamos los datos
for _, row in data.iterrows():
    cursor.execute(insert_query, (
        row['Fecha'],
        float(row['Minutos pospago salientes (miles)']),
        float(row['Minutos prepago salientes (miles)']),
        float(row['Total de minutos salientes (miles)'])
    ))

# Confirmamos los cambios y cerramos la conexión
connection.commit()
cursor.close()
connection.close()

print("Datos insertados correctamente en la tabla 'Minutos_salientes_Tel_Movil'.")


Datos insertados correctamente en la tabla 'Minutos_salientes_Tel_Movil'.


Cargamos los datos de la tabla mysql Telefonia_Movil_Penetracion con los datos del archivo  Penetracion.csv       

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones"
}

# Cargamos la ruta del archivo CSV
csv_file_path = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\Telefonia_movil\\Penetracion.csv'

# Cargamos el archivo CSV
data = pd.read_csv(csv_file_path)

# Establesemos la conexión a la base de datos
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

# Query para insertar datos
insert_query = """
INSERT INTO Telefonia_Movil_Penetracion (
    Fecha,
    Accesos_por_cada_100_hab
) VALUES (%s, %s)
"""

# Iteramos sobre el DataFrame e insertamos los datos
for _, row in data.iterrows():
    cursor.execute(insert_query, (
        row['Fecha'],
        float(row['Accesos por cada 100 hab'])
    ))

# Confirmamos los cambios y cerramos la conexión
connection.commit()
cursor.close()
connection.close()

print("Datos insertados correctamente en la tabla 'Telefonia_Movil_Penetracion'.")


Datos insertados correctamente en la tabla 'Telefonia_Movil_Penetracion'.


Cargamos los datos de la tabla mysql Accesos_provincias_TV  con los datos del archivo  Accesos_prov_TV.csv       

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "Tecnologias_comunicaciones"
}

# Ruta del archivo CSV
csv_file_path = r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Television\Accesos_prov_TV.csv'

# Cargamos el archivo CSV
data = pd.read_csv(csv_file_path)

# Establesemos la conexión a la base de datos
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

# Query para insertar datos
insert_query = """
INSERT INTO Accesos_provincias_TV (
    Id_Provincia,
    Provincia,
    Fecha,
    Accesos_TV_por_suscripcion
) VALUES (%s, %s, %s, %s)
"""

# Iteramos sobre el DataFrame e insertamos los datos
for _, row in data.iterrows():
    cursor.execute(insert_query, (
        int(row['Id_Provincia']),
        row['Provincia'],
        row['Fecha'],
        int(row['Accesos TV por suscripción (Vrf)'])
    ))

# Confirmamos los cambios y cerramos la conexión
connection.commit()
cursor.close()
connection.close()

print("Datos insertados correctamente en la tabla 'Accesos_provincias_TV'.")


Datos insertados correctamente en la tabla 'Accesos_provincias_TV'.


Cargamos los datos de la tabla mysql Accesos_totales_TV con los datos del archivo    Accesos_totales_TV.csv     

In [ ]:
# Configuramos la conexión a MySQL
db_config = {
    "host": "",
    "user": "r",
    "password": "",
    "database": "Tecnologias_comunicaciones"
}

# Ruta del archivo CSV
csv_file_path = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\Television\\Accesos_totales_TV.csv'

# Cargamos el archivo CSV
try:
    data = pd.read_csv(csv_file_path)
    print("Datos cargados correctamente:")
    print(data.head())
except Exception as e:
    print(f"Error al cargar el archivo CSV: {e}")
    exit()

# Esatblesemos la conexión a la base de datos
try:
    connection = mysql.connector.connect(**db_config)
    cursor = connection.cursor()
    print("Conexión exitosa a la base de datos.")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")
    exit()

# Query para insertar datos
insert_query = """
INSERT INTO Accesos_totales_TV (
    Fecha,
    Accesos_TV_por_suscripcion,
    Accesos_TV_satelital
) VALUES (%s, %s, %s)
"""

# Iteramos sobre el DataFrame e insertamos los datos
try:
    for _, row in data.iterrows():
        try:
            cursor.execute(insert_query, (
                row['Fecha'],
                int(row['Accesos TV por suscripción']),
                int(row['Accesos TV satelital'])
            ))
        except Exception as e:
            print(f"Error al insertar fila {row}: {e}")
    connection.commit()
    print("Datos insertados correctamente en la tabla 'Accesos_totales_TV'.")
except Exception as e:
    print(f"Error durante la inserción: {e}")

# Cerramos la conexión
try:
    cursor.close()
    connection.close()
    print("Conexión cerrada.")
except Exception as e:
    print(f"Error al cerrar la conexión: {e}")


Datos cargados correctamente:
        Fecha  Accesos TV por suscripción  Accesos TV satelital
0  2024-06-30                     7454428               1367489
1  2024-03-31                     7601499               1445205
2  2023-12-31                     7650809               1508789
3  2023-09-30                     7743735               1683205
4  2023-06-30                     8076400               1735655
Conexión exitosa a la base de datos.
Datos insertados correctamente en la tabla 'Accesos_totales_TV'.
Conexión cerrada.


Cargamos los datos de la tabla mysql Ingresos_TV con los datos del archivo   Ingresos_TV.csv      

In [ ]:
# Configuramos la  conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones"
}

# Ruta del archivo CSV
csv_file_path = r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Television\Ingresos_TV.csv'

# Cargamos el archivo CSV
try:
    data = pd.read_csv(csv_file_path)
    print("Datos cargados correctamente:")
    print(data.head())
except Exception as e:
    print(f"Error al cargar el archivo CSV: {e}")
    exit()

# Extablesemos la conexión a la base de datos
try:
    connection = mysql.connector.connect(**db_config)
    cursor = connection.cursor()
    print("Conexión exitosa a la base de datos.")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")
    exit()

# Query para insertar datos
insert_query = """
INSERT INTO Ingresos_TV (
    Fecha,
    Ingresos_TV_por_suscripcion,
    Ingresos_TV_satelital
) VALUES (%s, %s, %s)
ON DUPLICATE KEY UPDATE
    Ingresos_TV_por_suscripcion = VALUES(Ingresos_TV_por_suscripcion),
    Ingresos_TV_satelital = VALUES(Ingresos_TV_satelital);
"""

# Iteramos sobre el DataFrame e insertamos los datos
try:
    for _, row in data.iterrows():
        cursor.execute(insert_query, (
            row['Fecha'],
            float(row['Ingresos TV por suscripción  (miles de $)']),
            float(row['Ingresos TV satelital (miles de $)'])
        ))
    connection.commit()
    print("Datos insertados correctamente en la tabla 'Ingresos_TV'.")
except Exception as e:
    print(f"Error al insertar los datos: {e}")

# Cerramos la conexión
try:
    cursor.close()
    connection.close()
    print("Conexión cerrada.")
except Exception as e:
    print(f"Error al cerrar la conexión: {e}")


Datos cargados correctamente:
        Fecha  Ingresos TV por suscripción  (miles de $)  \
0  2024-06-30                               2.662339e+08   
1  2024-03-31                               1.910639e+08   
2  2023-12-31                               1.335822e+08   
3  2023-09-30                               1.037380e+08   
4  2023-06-30                               8.701537e+07   

   Ingresos TV satelital (miles de $)  
0                        1.248149e+08  
1                        9.427053e+07  
2                        6.474882e+07  
3                        5.117948e+07  
4                        3.862515e+07  
Conexión exitosa a la base de datos.
Datos insertados correctamente en la tabla 'Ingresos_TV'.
Conexión cerrada.


Cargamos los datos de la tabla mysql Penetracion_totales_TV con los datos del archivo  Penetracion_totales_TV.csv       

In [ ]:
# Configuramos la  conexión a MySQL
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "Tecnologias_comunicaciones"
}

# Ruta del archivo CSV
csv_file_path = r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\Television\Penetracion_totales_TV.csv'

# Cargamos el archivo CSV
try:
    data = pd.read_csv(csv_file_path)
    print("Datos cargados correctamente:")
    print(data.head())
except Exception as e:
    print(f"Error al cargar el archivo CSV: {e}")
    exit()

# Establesemos la conexión a la base de datos
try:
    connection = mysql.connector.connect(**db_config)
    cursor = connection.cursor()
    print("Conexión exitosa a la base de datos.")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")
    exit()

# Query para insertar datos
insert_query = """
INSERT INTO Penetracion_totales_TV (
    Fecha,
    Accesos_TV_por_suscripcion_por_100_hab,
    Accesos_TV_satelital_por_100_hab,
    Accesos_TV_por_suscripcion_por_100_hogares,
    Accesos_TV_satelital_por_100_hogares
) VALUES (%s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    Accesos_TV_por_suscripcion_por_100_hab = VALUES(Accesos_TV_por_suscripcion_por_100_hab),
    Accesos_TV_satelital_por_100_hab = VALUES(Accesos_TV_satelital_por_100_hab),
    Accesos_TV_por_suscripcion_por_100_hogares = VALUES(Accesos_TV_por_suscripcion_por_100_hogares),
    Accesos_TV_satelital_por_100_hogares = VALUES(Accesos_TV_satelital_por_100_hogares);
"""

# Iteramos sobre el DataFrame e insertamos los datos
try:
    for _, row in data.iterrows():
        cursor.execute(insert_query, (
            row['Fecha'],
            float(row['Accesos TV por suscripción por cada 100 hab']),
            float(row['Accesos TV satelital por cada 100 hab']),
            float(row['Accesos TV por suscripción por cada 100 hogares']),
            float(row['Accesos TV satelital por cada 100 hogares'])
        ))
    connection.commit()
    print("Datos insertados correctamente en la tabla 'Penetracion_totales_TV'.")
except Exception as e:
    print(f"Error al insertar los datos: {e}")

# Cerramos la conexión
try:
    cursor.close()
    connection.close()
    print("Conexión cerrada.")
except Exception as e:
    print(f"Error al cerrar la conexión: {e}")


Datos cargados correctamente:
        Fecha  Accesos TV por suscripción por cada 100 hab  \
0  2024-06-30                                        15.85   
1  2024-03-31                                        16.20   
2  2023-12-31                                        16.34   
3  2023-09-30                                        16.57   
4  2023-06-30                                        17.32   

   Accesos TV satelital por cada 100 hab  \
0                                   2.91   
1                                   3.08   
2                                   3.22   
3                                   3.60   
4                                   3.72   

   Accesos TV por suscripción por cada 100 hogares  \
0                                            50.40   
1                                            51.55   
2                                            52.05   
3                                            52.84   
4                                            55.29   

   Acce

Cargamos los datos de la tabla mysql  con los datos del archivo         